In [1]:
import re
import docx
import pandas as pd
import spacy
import skweak

In [2]:
# Funcao parar extrair contratos do .docx
def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return fullText

In [3]:
texto = getText('dodf43-62.docx')
# for txt in texto:
#   print(txt)

In [4]:
# Funcao parar extrair contratos do .txt
def resgate():
    pdf = open('contratos.txt', 'r')
    leitura = pdf.read().replace("\n", "")
    regex_ = re.compile(r"(EXTRATO\sD[O|E]\sCONTRATO[.|\s|\S]*?<>END OF BLOCK<>)")
    return(regex_.findall(leitura))

dados = resgate()
#dados

In [5]:
# Unindo os datasets
for txt in texto:
    dados.append(txt)
#dados    

In [6]:
# Carregando nlp do spacy em pt-br sem o ner e o lemmatizer

nlp = spacy.load('pt_core_news_sm', disable=["ner", "lemmatizer"])
docs = list(nlp.pipe(dados))

In [7]:
# Label functions Vitor Araruna
def contrato_(doc):
    expression = r"(\d+/\d{4})"
    for match in re.finditer(expression, doc.text): #doc.tex with m re.finditer to return span with char_span
        if(match.groups):
            grupo = (match.groups()[0])
            tamanho = len(grupo.split())
            for token in docs:
                if token.text in {grupo}:
                    yield token.i, token.i+tamanho, "CONTRATO"
                    # print(docs[0][token.i:token.i+tamanho])
            break 

def processo_(doc):
    expression = r"[P|p][R|r][O|o][C|c][E|e][S|s][S|s][O|o][:|\sSEI].*?(\d+[\.|-|–|\/]\d+[\.|-|–|\/]\d+[\.|-|–|\/]\d*)"
    for match in  re.finditer(expression, doc.text): #doc.tex with m re.finditer to return span with char_span
        if(match.groups):
            grupo = (match.groups()[0])
            grupo_copy = grupo
            if ("(" or ")") in grupo:
                    grupo = grupo.replace("(", "\(")
                    grupo = grupo.replace(")", "\)")
            tamanho = len(grupo.split())
            start = re.search(grupo, doc.text).span()[0]
            end = re.search(grupo, doc.text).span()[1]
            span = str(doc.char_span(start, end))
            x = re.findall(r'\/\s', span)
            for token in doc:
                if(grupo_copy in str(doc[token.i: token.i+tamanho+len(x)])):
                    yield token.i, token.i+tamanho+len(x), "PROCESSO"
                    break
            break

def data_assinatura_(doc):
    expression = r"[A|a][S|s][S|s][I|i][N|n][A|a][T|t][U|u][R|r][A|a]:.*?[\s\S](\d{2}\/\d{2}\/\d{4}|\d+.*?[\s\S]\w+.*?[\s\S]\w+.*?[\s\S]\w+.*?[\s\S]\d+)"
    for match in  re.finditer(expression, doc.text): #doc.tex with m re.finditer to return span with char_span
        if(match.groups):
            grupo = (match.groups()[0])
            grupo_copy = grupo
            if ("(" or ")") in grupo:
                    grupo = grupo.replace("(", "\(")
                    grupo = grupo.replace(")", "\)")
            tamanho = len(grupo.split())
            start = re.search(grupo, doc.text).span()[0]
            end = re.search(grupo, doc.text).span()[1]
            span = str(doc.char_span(start, end))
            x = re.findall(r'\/\s', span)
            for token in doc:
                if(grupo_copy in str(doc[token.i: token.i+tamanho+len(x)])):
                    yield token.i, token.i+tamanho+len(x), "DATA ASS."
                    break
            break

# def partes_(doc):
#     expression = r"[P|p][A|a][R|r][T|t][E|e][S|s]:.?([^,|;|]*)"
#     for match in re.finditer(expression, doc.text): #doc.tex with m re.finditer to return span with char_span
#         if(match.groups):
#             grupo = (match.groups()[0])
#             if ("(" or ")") in grupo:
#                     grupo = grupo.replace("(", "\(")
#                     grupo = grupo.replace(")", "\)")
#             start = re.search(grupo, docs[0].text).span()[0]
#             end = re.search(grupo, docs[0].text).span()[1]
#             span = docs[0].char_span(start, end)
#             print("Found match:", span.text)
#             break

# def objeto_(doc):
#     expression = r"[O|o][B|b][J|j][E|e][T|t][O|o]:.?([^,|;|.]*)"
#     for match in re.finditer(expression, doc.text): #doc.tex with m re.finditer to return span with char_span
#         if(match.groups):
#             grupo = (match.groups()[0])
#             if ("(" or ")") in grupo:
#                     grupo = grupo.replace("(", "\(")
#                     grupo = grupo.replace(")", "\)")
#             start = re.search(grupo, docs[0].text).span()[0]
#             end = re.search(grupo, docs[0].text).span()[1]
#             span = docs[0].char_span(start, end)
#             print("Found match:", span.text)
#             break


def vigencia_(doc):
    expression = r"[V|v][I|i][G|g][E|e|ê][N|n][C|c][I|i][A|a].*?[\S\s].*?([^,|;|.]*)"
    for match in  re.finditer(expression, doc.text): #doc.tex with m re.finditer to return span with char_span
        if(match.groups):
            grupo = (match.groups()[0])
            grupo_copy = grupo
            if ("(" or ")") in grupo:
                    grupo = grupo.replace("(", "\(")
                    grupo = grupo.replace(")", "\)")
            tamanho = len(grupo.split())
            start = re.search(grupo, doc.text).span()[0]
            end = re.search(grupo, doc.text).span()[1]
            span = str(doc.char_span(start, end))
            x = re.findall(r'[\(|\)]', span)
            for token in doc:
                if(grupo_copy in str(doc[token.i: token.i+tamanho+len(x)])):
                    yield token.i, token.i+tamanho+len(x), "VIGENCIA"
                    break
            break

def valor_(doc):
    expression = r"[v|V][a|A][l|L][o|O][r|R].*?[\s\S].*?([\d\.]*,\d{2})"
    for match in  re.finditer(expression, doc.text): #doc.tex with m re.finditer to return span with char_span
        if(match.groups):
            grupo = (match.groups()[0])
            grupo_copy = grupo
            if ("(" or ")") in grupo:
                    grupo = grupo.replace("(", "\(")
                    grupo = grupo.replace(")", "\)")
            tamanho = len(grupo.split())
            start = re.search(grupo, doc.text).span()[0]
            end = re.search(grupo, doc.text).span()[1]
            span = str(doc.char_span(start, end))
            x = re.findall(r'[\(|\)]', span)
            for token in doc:
                if(grupo_copy in str(doc[token.i: token.i+tamanho+len(x)])):
                    yield token.i-1, token.i+tamanho+len(x), "VALOR"
                    break
            break

def unidade_orcamento_(doc):
    expression = r"[o|O][r|R][c|C|ç|Ç][a|A][m|M][e|E][n|N][t|T][a|A|á|Á][r|R][i|I][a|A].*?[\s\S].*?(\d+.\d+)"
    for match in  re.finditer(expression, doc.text): #doc.tex with m re.finditer to return span with char_span
        if(match.groups):
            grupo = (match.groups()[0])
            grupo_copy = grupo
            if ("(" or ")") in grupo:
                    grupo = grupo.replace("(", "\(")
                    grupo = grupo.replace(")", "\)")
            tamanho = len(grupo.split())
            start = re.search(grupo, doc.text).span()[0]
            end = re.search(grupo, doc.text).span()[1]
            span = str(doc.char_span(start, end))
            x = re.findall(r'[\(|\)]', span)
            for token in doc:
                if(grupo_copy in str(doc[token.i: token.i+tamanho+len(x)])):
                    yield token.i, token.i+tamanho+len(x), "UNI ORC."
                    break
            break

def programa_trabalho_(doc):
    expression = r"[T|t][R|r][A|a][B|b][A|a][L|l][H|h][O|o]?[:|;|[\s\S].*?(\d*\.\d*.\d*.\d*.\d{4,6}.\d{4,6})"
    for match in  re.finditer(expression, doc.text): #doc.tex with m re.finditer to return span with char_span
        if(match.groups):
            grupo = (match.groups()[0])
            grupo_copy = grupo
            if ("(" or ")") in grupo:
                    grupo = grupo.replace("(", "\(")
                    grupo = grupo.replace(")", "\)")
            tamanho = len(grupo.split())
            start = re.search(grupo, doc.text).span()[0]
            end = re.search(grupo, doc.text).span()[1]
            span = str(doc.char_span(start, end))
            x = re.findall(r'[\(|\)]', span)
            for token in doc:
                if(grupo_copy in str(doc[token.i: token.i+tamanho+len(x)])):
                    yield token.i, token.i+tamanho+len(x), "PROG TRAB."
                    break
            break

def natureza_emepnho_(doc):
    expression = r"Natureza d[e|a] Despesa:[\s\S].*?([\d.\d]*)"
    for match in  re.finditer(expression, doc.text): #doc.tex with m re.finditer to return span with char_span
        if(match.groups):
            grupo = (match.groups()[0])
            grupo_copy = grupo
            if ("(" or ")") in grupo:
                    grupo = grupo.replace("(", "\(")
                    grupo = grupo.replace(")", "\)")
            tamanho = len(grupo.split())
            start = re.search(grupo, doc.text).span()[0]
            end = re.search(grupo, doc.text).span()[1]
            span = str(doc.char_span(start, end))
            x = re.findall(r'[\(|\)]', span)
            for token in doc:
                if(grupo_copy in str(doc[token.i: token.i+tamanho+len(x)])):
                    yield token.i, token.i+tamanho+len(x), "NAT DESP."
                    break
            break

def nota_empenho_(doc):
    expression = r"(\d+NE\d+)"
    for match in  re.finditer(expression, doc.text): #doc.tex with m re.finditer to return span with char_span
        if(match.groups):
            grupo = (match.groups()[0])
            grupo_copy = grupo
            if ("(" or ")") in grupo:
                    grupo = grupo.replace("(", "\(")
                    grupo = grupo.replace(")", "\)")
            tamanho = len(grupo.split())
            start = re.search(grupo, doc.text).span()[0]
            end = re.search(grupo, doc.text).span()[1]
            span = str(doc.char_span(start, end))
            x = re.findall(r'[\(|\)]', span)
            for token in doc:
                if(grupo_copy in str(doc[token.i: token.i+tamanho+len(x)])):
                    yield token.i, token.i+tamanho+len(x), "NOTA EMP."
                    break
            break


# def entidade_contratada_(doc):
#     expression = r"[p|P][e|E][l|L][a|A|o|O].*?[\S\s].*?[c|C][o|O][n|N][t|T][r|R][a|A][t|T][a|A][d|D][a|A|o|O].*?[\S\s].*?(\w+[\s\S].*?[\s\S].*?[\.])"
#     for match in re.finditer(expression, doc.text): #doc.tex with m re.finditer to return span with char_span
#         if(match.groups):
#             grupo = (match.groups()[0])
#             tamanho = len(grupo.split())
#             start = re.search(grupo, doc.text).span()[0]
#             end = re.search(grupo, doc.text).span()[1]
#             span = str(doc.char_span(start, end))
#             x = re.findall(r'[\.|,|:|;]', span)
#             for token in doc:
#                 if(grupo in str(doc[token.i: token.i+tamanho+len(x)])):
#                     yield token.i, token.i+tamanho+len(x), "ENT. CONTRATADA"
#                     print(doc[token.i:token.i+tamanho+len(x)])
#                     break
#             break



In [8]:
# Aplicando cada uma no dataset
detec_contrato = skweak.heuristics.FunctionAnnotator("detec_contrato", contrato_)
docs = list(detec_contrato.pipe(docs))

detec_processo = skweak.heuristics.FunctionAnnotator("detec_processo", processo_)
docs = list(detec_processo.pipe(docs))

detec_data = skweak.heuristics.FunctionAnnotator("detec_data", data_assinatura_)
docs = list(detec_data.pipe(docs))

detec_vigencia = skweak.heuristics.FunctionAnnotator("detec_vigencia", vigencia_)
docs = list(detec_vigencia.pipe(docs))

detec_valor = skweak.heuristics.FunctionAnnotator("detec_valor", valor_)
docs = list(detec_valor.pipe(docs))

detec_unidade = skweak.heuristics.FunctionAnnotator("detec_unidade", unidade_orcamento_)
docs = list(detec_unidade.pipe(docs))

detec_programa = skweak.heuristics.FunctionAnnotator("detec_programa", programa_trabalho_)
docs = list(detec_programa.pipe(docs))

detec_natureza = skweak.heuristics.FunctionAnnotator("detec_natureza", natureza_emepnho_)
docs = list(detec_natureza.pipe(docs))

detec_nota = skweak.heuristics.FunctionAnnotator("detec_nota", nota_empenho_)
docs = list(detec_nota.pipe(docs))

In [48]:
# Label functions Vitor Vasconcelos
vet = {'Partes','PARTES','partes:','Objeto','OBJETO','Valor','VALOR','Valor:','VALOR:','valor:','Lei','LEI','lei:','Lei:','LEI:','Dotação','DOTAÇÃO','dotação','DOTACAO','Dotacao','dotacao','Unidade','UNIDADE','unidade','Programa','PROGRAMA','programa','Natureza','NATUREZA','natureza','Data','DATA','data','Assinatura','ASSINATURA','assinatura:','Assinatura:','ASSINATURA:','SIGNATÁRIOS','SIGNATARIOS','Signatários:','SIGNATÁRIOS:','SIGNATARIOS:','Signatarios:','Signatarios','Assinantes','ASSINANTES','Assinantes:','ASSINANTES:','<>END OF BLOCK<>','END OF BLOCK'}

vet2 = {'Partes','PARTES','partes:','Objeto','OBJETO','Valor','VALOR','Valor:','VALOR:','valor:','Lei','LEI','lei:','Lei:','LEI:','Dotação','DOTAÇÃO','dotação','DOTACAO','Dotacao','dotacao','Unidade','UNIDADE','unidade','Programa','PROGRAMA','programa','Natureza','NATUREZA','natureza','Data','DATA','data','Assinatura','ASSINATURA','assinatura:','Assinatura:','ASSINATURA:','<>END OF BLOCK<>','END OF BLOCK'}

vet3 = {'Objeto','OBJETO','Valor','VALOR','Valor:','VALOR:','valor:','Lei','LEI','lei:','Lei:','LEI:','Dotação','DOTAÇÃO','dotação','DOTACAO','Dotacao','dotacao','Unidade','UNIDADE','unidade','Programa','PROGRAMA','programa','Natureza','NATUREZA','natureza','Data','DATA','data','Assinatura','ASSINATURA','assinatura:','Assinatura:','ASSINATURA:','<>END OF BLOCK<>','END OF BLOCK'}

def contrato_detector_fun(doc):
    for token in doc:
        if token.text in {'CONTRATO'}:
            for x in range((len(doc)-token.i-1)):
                if doc[token.i+x].text in {'No','NO','no','Nº','nº','N°','n°'} and doc[token.i+x+1].text[0].isdigit():
                    yield doc[token.i+x].i, doc[token.i+x].i+2, "CONTRATO",
                    break

def processo_detector_fun(doc):
    for token in doc:                
        for y in ['Processo','PROCESSO','Processo:','PROCESSO:']:
            if y in token.text:
                k=0
                if(doc[token.i+1].text==':'):
                    k=1
                for x in range((len(doc)-token.i)):
                    if doc[token.i+x].text in {'.','-','–',',',';','('} or doc[token.i+x].text in vet:
                        yield token.i+1+k, token.i+x, "PROCESSO"
                        break           

def partes_detector_fun(doc):
    for token in doc:  
        for y in ['Partes','PARTES','partes:']:
            if y in token.text:           
                k=0
                if(doc[token.i+1].text==':'):
                    k=1
                for x in range((len(doc)-token.i)):
                    if doc[token.i+x].text in {'.',';'} or doc[token.i+x].text in vet3:
                        yield token.i+1+k, token.i+x, "PARTES"
                        break

def objeto_detector_fun(doc):
    flag=0
    for token in doc:    
        if token.text in {'Objeto','OBJETO'}:
            k=0
            if(doc[token.i+1].text==':'):
                k=1
            for x in range((len(doc)-token.i-1)):
                if doc[token.i+x+1].text in vet:#{'.'}:
                    flag=1
                    yield token.i+1+k, token.i+x-1, "OBJETO"
                    break
        if token.text in {'tem','Tem'} and doc[token.i+1].text in {'por','POR'} and doc[token.i+2].text in {'objeto','Objeto','OBJETO'}  and flag==0:
            for x in range((len(doc)-token.i-1)):
                if doc[token.i+x+1].text in vet:#{'.'}:
                    yield token.i, token.i+x, "OBJETO"
                    break
                            
def valor_detector_fun(doc):
    for token in doc:    
        for y in ['Valor','VALOR','Valor:','VALOR:','valor:']:
            if y in token.text:
                for x in range((len(doc)-token.i-1)):
                    if doc[token.i+x].text in {'R$','$','$$'} and doc[token.i+x+1].text[0].isdigit():
                        yield doc[token.i+x].i, doc[token.i+x].i+2, "VALOR",
                        break


def lei_orc_detector_fun(doc):
    for token in doc:    
        for y in ['Lei','LEI','lei:','Lei:','LEI:']:
            if y in token.text and doc[token.i+1].text in {'Orçamentária','ORÇAMENTÁRIA','ORÇAMENTARIA','oçamentária','Orçamentária:','ORÇAMENTÁRIA:','ORÇAMENTARIA:','oçamentária:'}:
                k=0
                if(doc[token.i+1].text==':'):
                    k=1
                for x in range((len(doc)-token.i)):
                    if doc[token.i+x].text in {'.','-','–',',',';','('}:
                        yield token.i+2+k, token.i+x, "LEI ORC."
                        break

def dotacao_orc_detector_fun(doc):
    for token in doc:  
        for y in ['Dotação','DOTAÇÃO','dotação','DOTACAO','Dotacao','dotacao']: 
            if y in token.text and doc[token.i+1].text in {'Orçamentária','ORÇAMENTÁRIA','ORÇAMENTARIA','oçamentária','Orçamentária:','ORÇAMENTÁRIA:','ORÇAMENTARIA:','oçamentária:'}:
                k=0
                if(doc[token.i+1].text==':'):
                    k=1
                for x in range((len(doc)-token.i)):
                    if doc[token.i+x].text in {'.','-','–',',',';','('}:
                        yield token.i+3, token.i+x, "DOT ORC."
                        break

def unidade_orc_detector_fun(doc):
    for token in doc: 
        for y in ['Unidade','UNIDADE','unidade']: 
            if y in token.text and doc[token.i+1].text in {'Orçamentária','Orcamentaria','ORÇAMENTÁRIA','ORCAMENTARIA','orcamentaria','orçamentária','Orçamentária:','Orcamentaria:','ORÇAMENTÁRIA:','ORCAMENTARIA:','orcamentaria:','orçamentária:'}:
                k=0
                if(doc[token.i+2].text==':'):
                    k=1
                for x in range((len(doc)-token.i)):
                    if doc[token.i+x].text in {'.','-','–',',',';','('}:
                        yield token.i+2+k, token.i+x, "UNI ORC."
                        break

def programa_trab_detector_fun(doc):
    for token in doc: 
        for y in ['Programa','PROGRAMA','programa']:   
            if y in token.text and doc[token.i+1].text in {'de','do','da','DE','DO','DA'} and doc[token.i+2].text in {'trabalho','Trabalho','TRABALHO','trabalho:','Trabalho:','TRABALHO:'}:
                k=0
                if(doc[token.i+3].text==':'):
                    k=1
                for x in range((len(doc)-token.i)):
                    if doc[token.i+x].text in {'.','-','–',',',';','('}:
                        yield token.i+3+k, token.i+x, "PROG TRAB."
                        break

def natureza_desp_detector_fun(doc):
    for token in doc:    
        for y in ['Natureza','NATUREZA','natureza']:  
            if y in token.text and doc[token.i+1].text in {'de','do','da','DE','DO','DA','DAS','das'} and doc[token.i+2].text in {'despesa','Despesa','DESPESA','despesa:','Despesa:','DESPESA:','despesas','Despesas','DESPESAs','despesas:','Despesas:','DESPESAs:'}:
                k=0
                if(doc[token.i+3].text==':'):
                    k=1
                for x in range((len(doc)-token.i)):
                    if doc[token.i+x].text in {'.','-','–',',',';','('}:
                        yield token.i+3+k, token.i+x, "NAT DESP."
                        break

def data_detector_fun(doc):
    for token in doc:  
        for y in ['Assinatura','ASSINATURA','assinatura:','Assinatura:','ASSINATURA:']:  
            if y in token.text:
                k=0
                if(doc[token.i+1].text==':'):
                    k=1  
                for x in range((len(doc)-token.i)):
                    if doc[token.i+x].text in {'.','-','–',',',';','(','Partes','PARTES','partes:','Objeto','OBJETO','Valor','VALOR','Valor:','VALOR:','valor:','Lei','LEI','lei:','Lei:','LEI:','Dotação','DOTAÇÃO','dotação','DOTACAO','Dotacao','dotacao','Unidade','UNIDADE','unidade','Programa','PROGRAMA','programa','Natureza','NATUREZA','natureza','SIGNATÁRIOS','SIGNATARIOS','Signatários:','SIGNATÁRIOS:','SIGNATARIOS:','Signatarios:','Signatarios','Assinantes','ASSINANTES','Assinantes:','ASSINANTES:','<>END OF BLOCK<>','END OF BLOCK'}:
                        yield token.i+1+k, token.i+x, "DATA ASS."
                        break


def signatarios_detector_fun(doc):
    for token in doc:  
        for y in ['SIGNATÁRIOS','SIGNATARIOS','Signatários:','SIGNATÁRIOS:','SIGNATARIOS:','Signatarios:','Signatarios','Assinantes','ASSINANTES','Assinantes:','ASSINANTES:']: 
            if y in token.text:
                k=0
                if(doc[token.i+1].text==':'):
                    k=1 
                for x in range((len(doc)-token.i-2)):
                    if '.' in doc[token.i+x+2].text :
                        yield token.i+1+k, token.i+x+3, "SIGNATARIOS"
                        break

def vigencia_detector_fun(doc):
    for token in doc:    
        for y in ['VIGÊNCIA','VIGENCIA','Vigência:','VIGÊNCIA:','VIGENCIA:','Vigencia:','Vigencia']:#'Vigencia','Vigência'
            if y in token.text:
                k=0
                if(doc[token.i+1].text==':'):
                    k=1
                for x in range((len(doc)-token.i)):
                    if doc[token.i+x].text in {'.','-','–',';'}:
                        yield token.i+1+k, token.i+x, "VIGENCIA"
                        break

def nota_emp_detector_fun(doc):
    for token in doc: 
        for y in ['Empenho','EMPENHO','Empenho:','EMPENHO:','empenho:']:   
            if y in token.text:
                k=0
                if(doc[token.i+1].text==':'):
                    k=1 
                for x in range((len(doc)-token.i)):
                    if doc[token.i+x].text in {'.','-','–',',',';','('}:
                        yield token.i+1+k, token.i+x, "NOTA EMP."
                        break
                
def numero_ajuste_detector_fun(doc):
    for token in doc:
        for y in ['número','numero','Número','Numero','NÚMERO','NUMERO']:     
            if y in token.text and doc[token.i+1].text in {'de','do','da','DE','DO','DA'} and doc[token.i+2].text in {'Ajuste','AJUSTE','ajuste','Ajuste:','AJUSTE:','ajuste:'}:
                k=0
                if(doc[token.i+1].text==':'):
                    k=1
                for x in range((len(doc)-token.i)):
                    if doc[token.i+x].text in {'.','-','–',',',';','('}:
                        yield token.i+3+k, token.i+x, "NUM AJUSTE."
                        break

def procedimento_detector_fun(doc):
    for token in doc:    
        for y in ['Procedimento','PROCEDIMENTO','PROCEDIMENTO:','procedimento:','Procedimento:']:
            if y in token.text:
                k=0
                if(doc[token.i+1].text==':'):
                    k=1
                for x in range((len(doc)-token.i)):
                    if doc[token.i+x].text in vet:
                        yield token.i+1+k, token.i+x, "PROCED."
                        break


def money_detector_fun(doc):
    for token in doc:
        if token.text[0].isdigit() and doc[token.i-1].text in {'R$','$','$$'}:
            yield token.i-1, token.i+1, "DINHEIRO"


In [51]:
# Aplicando cada uma no dataset
contrato_detector = skweak.heuristics.FunctionAnnotator("contrato_detector", contrato_detector_fun)
docs = list(contrato_detector.pipe(docs))

processo_detector = skweak.heuristics.FunctionAnnotator("processo_detector", processo_detector_fun)
docs = list(processo_detector.pipe(docs))

partes_detector = skweak.heuristics.FunctionAnnotator("partes_detector", partes_detector_fun)
docs = list(partes_detector.pipe(docs))

objeto_detector = skweak.heuristics.FunctionAnnotator("objeto_detector", objeto_detector_fun)
docs = list(objeto_detector.pipe(docs))

valor_detector = skweak.heuristics.FunctionAnnotator("valor_detector", valor_detector_fun)
docs = list(valor_detector.pipe(docs))

lei_orc_detector = skweak.heuristics.FunctionAnnotator("lei_orc_detector", lei_orc_detector_fun)
docs = list(lei_orc_detector.pipe(docs))

unidade_orc_detector = skweak.heuristics.FunctionAnnotator("unidade_orc_detector", unidade_orc_detector_fun)
docs = list(unidade_orc_detector.pipe(docs))

programa_trab_detector = skweak.heuristics.FunctionAnnotator("programa_trab_detector", programa_trab_detector_fun)
docs = list(programa_trab_detector.pipe(docs))

natureza_desp_detector = skweak.heuristics.FunctionAnnotator("natureza_desp_detector", natureza_desp_detector_fun)
docs = list(natureza_desp_detector.pipe(docs))

data_detector = skweak.heuristics.FunctionAnnotator("data_detector", data_detector_fun)
docs = list(data_detector.pipe(docs))

signatarios_detector = skweak.heuristics.FunctionAnnotator("signatarios_detector", signatarios_detector_fun)
docs = list(signatarios_detector.pipe(docs))

vigencia_detector = skweak.heuristics.FunctionAnnotator("vigencia_detector", vigencia_detector_fun)
docs = list(vigencia_detector.pipe(docs))

nota_emp_detector = skweak.heuristics.FunctionAnnotator("nota_emp_detector", nota_emp_detector_fun)
docs = list(nota_emp_detector.pipe(docs))

numero_ajuste_detector = skweak.heuristics.FunctionAnnotator("numero_ajuste_detector", numero_ajuste_detector_fun)
docs = list(numero_ajuste_detector.pipe(docs))

procedimento_detector = skweak.heuristics.FunctionAnnotator("procedimento_detector", procedimento_detector_fun)
docs = list(procedimento_detector.pipe(docs))

money_detector = skweak.heuristics.FunctionAnnotator("money_detector", money_detector_fun)
docs = list(money_detector.pipe(docs))

In [56]:
# Funcoes "cruas" aplicadas
all_fun= ["contrato_detector","processo_detector","partes_detector","objeto_detector","valor_detector","lei_orc_detector","unidade_orc_detector","programa_trab_detector","natureza_desp_detector","data_detector","signatarios_detector","vigencia_detector","procedimento_detector","money_detector"]#,"money_detector"

skweak.utils.display_entities(docs[-1], all_fun)

In [53]:
# Calculo do algoritimo HMM e agregacao das funcoes
model = skweak.aggregation.HMM("hmm", ["CONTRATO", "PROCESSO", "PARTES", "OBJETO", "VALOR", "LEI ORC.", "DOT ORC.", "UNI ORC.", "PROG TRAB.", "NAT DESP.", "DATA ASS.", "SIGNATARIOS", "VIGENCIA","NOTA EMP.","NUM AJUSTE.","PROCED."],sequence_labelling = True) #,"DINHEIRO"

docs=model.fit_and_aggregate(docs)

Starting iteration 1
Finished E-step with 72 documents
Starting iteration 2


         1      -16460.9741             +nan


Finished E-step with 72 documents
Starting iteration 3


         2       -9336.6858       +7124.2883


Finished E-step with 72 documents
Starting iteration 4


         3       -9236.0985        +100.5874


Finished E-step with 72 documents
Starting iteration 5


         4       -9231.9370          +4.1615


Finished E-step with 72 documents


         5       -9230.8734          +1.0636


In [57]:
# Resultado do HMM aplicado
skweak.utils.display_entities(docs[-1], "hmm", add_tooltip=False)

In [14]:
# Salvando modelo HMM para treinamento 
for doc in docs:
    doc.ents = doc.spans["hmm"]
skweak.utils.docbin_writer(docs, "./data/reuters_small.spacy")

Write to ./data/reuters_small.spacy...done


In [15]:
# Mostrando entidades no modelo em IOB
from spacy.tokens import DocBin
nlp = spacy.blank("pt")
doc_bin = DocBin().from_disk("./data/reuters_small.spacy") 
examples = [] 

# for doc in doc_bin.get_docs(nlp.vocab):
#     label_iob = ""
#     for i in range(0, len(doc)):
#         _label_ = doc[i].ent_iob_
#         _ent_ = doc[i].ent_type_
#         if(_label_ != "O"):
#             label_iob+=f'{_label_}-{_ent_} '
#         else:
#             label_iob+=f'{_label_} '
#     spans = [{"start": ent.start_char, "end": ent.end_char, "label": ent.label_} for ent in doc.ents]
#     iob_f = [{"text":ent.text, "label": ent.label_} for ent in doc.ents]
#     examples.append({"text": doc.text, "iob": label_iob,"spans": spans,"IOB-F":iob_f})

# for doc in doc_bin.get_docs(nlp.vocab):
#     examples.append([{"ent":ent.text, "label": ent.label_} for ent in doc.ents])
#     #examples.append({"IOB-F":iob_f})

for doc in doc_bin.get_docs(nlp.vocab):
    lista_iob=[]
    for i in range(0, len(doc)):
        label_iob = ""
        _txt_ = doc[i].text
        _label_ = doc[i].ent_iob_
        _ent_ = doc[i].ent_type_
        if(_label_ != "O"):
            label_iob+=f'{_txt_} {_label_}-{_ent_}'
        else:
            label_iob+=f'{_txt_} {_label_}'
        lista_iob.append(label_iob)
    examples.append(lista_iob)
    

In [16]:
examples

[['EXTRATO O',
  'DO O',
  'CONTRATO O',
  'No B-CONTRATO',
  '29/2013 I-CONTRATO',
  ', O',
  'NOS O',
  'TERMOS O',
  'DO O',
  'PADRAO O',
  'No O',
  '09/2012.Processo O',
  ': B-PROCESSO',
  '139.000.604/2013 B-PROCESSO',
  '. O',
  'Partes O',
  ': O',
  'DF B-PARTES',
  '/ I-PARTES',
  'RAXI I-PARTES',
  'x I-PARTES',
  'SUMMIT I-PARTES',
  'CONSTRUCOES I-PARTES',
  'E I-PARTES',
  'TECNOLOGIA I-PARTES',
  'LTDA.-EPP I-PARTES',
  ', I-PARTES',
  'Fundamento I-PARTES',
  'Legal I-PARTES',
  ': I-PARTES',
  'Convite I-PARTES',
  'no I-PARTES',
  '028/ I-PARTES',
  '2013-RA I-PARTES',
  'XI I-PARTES',
  '. O',
  'Objeto O',
  ': O',
  'Contratacao B-OBJETO',
  'de I-OBJETO',
  'Empresa I-OBJETO',
  'para I-OBJETO',
  'Execucao I-OBJETO',
  'de I-OBJETO',
  'Reforma I-OBJETO',
  'e I-OBJETO',
  'Instalacoes I-OBJETO',
  'Eletrica I-OBJETO',
  'da I-OBJETO',
  'Feira I-OBJETO',
  'Permanente I-OBJETO',
  'do I-OBJETO',
  'Cruzeiro I-OBJETO',
  ', I-OBJETO',
  'conforme I-OBJETO',
  '

In [17]:
# treinamento do modelo final para aplicar no spacy

!spacy init config - --lang pt --pipeline ner --optimize accuracy | spacy train - --paths.train ./data/reuters_small.spacy  --paths.dev ./data/reuters_small.spacy --initialize.vectors pt_core_news_sm --output ./data/reuters_small

ℹ Saving to output directory: data\reuters_small
ℹ Using CPU



[2021-10-26 08:36:23,187] [INFO] Set up nlp object from config
[2021-10-26 08:36:23,245] [INFO] Pipeline: ['tok2vec', 'ner']
[2021-10-26 08:36:23,250] [INFO] Created vocabulary
[2021-10-26 08:36:27,832] [WARNING] [W112] The model specified to use for initial vectors (pt_core_news_sm) has no vectors. This is almost certainly a mistake.

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    137.00    0.00    0.00    0.00    0.00
  2     200       2804.47   9950.20   75.98   74.25   77.79    0.76
  5     400       1666.12   2162.90   84.69   86.52   82.94    0.85
  8     600        952.73   1046.70   95.81   97.04   94.60    0.96
 11     800       2729.32    794.97   95.19   94.78   95.61    0.95
 13    1000       4924.06    665.97   96.57   97.69   95.48    0.97
 16    1200       1097.29    424.39   97.50   97.02   97.99    0.98
 19    1400        103.98    241.12   97.79   98.60   96.99    0.98
 22    1600       3248.59    419.75   96.80   96.86   96.74    0.97
 25    1800  


[2021-10-26 08:36:27,834] [INFO] Added vectors: pt_core_news_sm
[2021-10-26 08:36:27,840] [INFO] Finished initializing nlp object
[2021-10-26 08:36:39,258] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


In [18]:
# Carregando nlp como nosso modelo criado
nlp = spacy.load("./data/reuters_small/model-best")

In [20]:
txt="EXTRATO DO CONTRATO Nº 13/2014 Processo 410.000.434/2014 – DAS PARTES:SEPLAN x SERVEGEL – APOIO ADMINISTRATIVO E SUPORTE OPERACIONAL LTDA. DO PROCEDIMENTO: O presente Contrato obedece aos termos do EDITAL DE LICITAÇÃO DE PREGÃO ELETRONICO N.º 151/ 2012 – SULIC/SEPLAN (fls. 03 a 149) oriundo do processo licitatório autuado sob nº 411.000.023/ 2012. DO OBJETO: O Contrato tem por objeto a contratação de empresa especializada para realizar a prestação de serviços continuados de limpeza asseio e conservação nos próprios do Governo do Distrito Federal com fornecimento de mão-de-obra materiais e equipamentos conforme especificações e condições estabelecidas no Anexo I do Edital LOTE 11 consoante especificam o EDITAL DE LICITAÇÃO DE PREGÃO ELETRONICO N.º 151/2012 – SULIC/SEPLAN (fls. 03 a 149) oriundo do processo licitatório autuado sob nº 411.000.023/2012. DO VALOR: O valor total do contrato é de R$ 5.645.716,32 >(cinco milhões seiscentos e quarenta e cinco mil setecentos e dezesseis reais e trinta e dois centavos). DA DOTAÇÃO ORÇAMENTÁRIA: A despesa correrá à conta da seguinte Dotação Orçamentária: I – Unidade Orçamentária: 32.101; II – Programa de Trabalho: 04.122.6003.2990.0006; III – Natureza da Despesa: 3.3.90.37; IV – Fonte de Recursos: 100; V – Nota de Empenho: 2014NE00621. DO PRAZO DE VIGÊNCIA: O contrato terá vigência de 12 (doze) meses a contar da data de sua assinatura. DA ASSINATURA: 08/05/2014. DOS SIGNATÁRIOS: Pela SEPLAN: Paulo Antenor de Oliveira na qualidade de Secretário de Estado e pela CONTRATADA: Marcelo Henry Soares Monteiro, na qualidade de Sócio-Proprietário da Empresa"

skweak.utils.display_entities(nlp(txt))

In [45]:
skweak.utils.display_entities(nlp(dados[5]))